*Updated 9/7/2021*

This is a tool for calculating playoff probabilities in a fantasy sports league that is (1) points-based and (2) head-to-head. It requires the following data:

1. A **scores** frame which contains each team's final score for each week of the league.
2. A **remaining schedule** frame.
3. A **current standings** frame containing current standings as well as any necessary information for a tiebreaker.

If your points system has a high probability for ties or a complex tiebreaker system you may want to augment this code to include that possibility. I will treat ties as 0.5 wins.

In all cases, I am outputting my versions of these files for reference in the code below.

In [3]:
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import collections
from functools import reduce


Two manual inputs for an average score (as a default mean) and a standard deviation of scores (for all estimations); this is almost certainly wider than the "true" standard deviation of a given scores, but I want to default to less confident estimates of win/loss where possible.

In [458]:
avg = 114.5
wobble = 28

Below estimates (for reference purposes) a %chance of victory based on projected score, with 28 being the standard deviation of scores from the 2020 season. (This figure will be subjectively adjusted as necessary.)

In [459]:
for k in range(0, 101, 10):
    j = 0
    for i in range(0,100000):
        x = np.random.normal(loc=114+k, scale=wobble)
        y = np.random.normal(loc=114, scale=wobble)
        if x > y:
            j = j + 1
    print(k, ' ', j/100000)

0   0.49752
10   0.59951
20   0.69324
30   0.77992
40   0.84534
50   0.89667
60   0.93559
70   0.96199
80   0.97845
90   0.98817
100   0.99417


Imports a frame of weekly point totals, and generates some reference variables for week of the season based on how filled in said frame is.

In [462]:
scores = pd.read_excel('UsacOneFantasyProps2021.xlsx', sheet_name='Scores', index_col='!Team') #updated dataframe of scores for week
team_list = list(scores.index) #list of teams
wk_count = len([i for i in list(scores.iloc[0]) if pd.notna(i) == True]) -1
print(wk_count)

0


Failsafes/confirming right file is fed in

In [463]:
wk_list = list(range(1, wk_count + 1)) #list of completed weeks as integers
wk_list = [str(i) for i in wk_list]
wk_list

[]

In [464]:
 scores.loc['The Attentive Listeners'] #reference

Total    0.0
Wk1      NaN
Wk2      NaN
Wk3      NaN
Wk4      NaN
Wk5      NaN
Wk6      NaN
Wk7      NaN
Wk8      NaN
Wk9      NaN
Wk10     NaN
Wk11     NaN
Wk12     NaN
Wk13     NaN
Wk14     NaN
Wk15     NaN
Wk16     NaN
Wk17     NaN
Name: The Attentive Listeners, dtype: float64

Generates projected scores for remaining weeks. Heading into week _n_:

* The mean projected score for week _n_ is the average of scores for weeks _n-1_ through _n-5_
* The mean projected score for week _n+1_ is the average of scores for weeks _n-1_ through _n-4_ and the average normalized score for all teams, to simulate regression to the mean over time
* In weeks where _n-1_ or further back does not exist, the average is also used

In [465]:
(1/bye_multiplier[wk_count+4])

1.0

In [466]:
bye_multiplier = [1,1,1,1,1,\
                  1.143,1.231,1.067,1.143,1.143,\
                  1.067,1.067,1.143, 1.143,1,\
                  1,1] #used to adjust for weeks in which less than 32 teams are playing


for team in team_list: #ADD IN BYE WEEK NONSENSE
    print(team)
    final_scores = list(scores.loc[team][1:wk_count+1])
    try:
        neg_one = final_scores[-1] * bye_multiplier[wk_count-1]
    except IndexError:
        if wk_count >= 1:
            neg_one = avg * bye_multiplier[wk_count-1]
        else:
            neg_one = avg
        
    try:
        neg_two = final_scores[-2] * bye_multiplier[wk_count-2]
    except IndexError:
        if wk_count >= 2:
            neg_two = avg * bye_multiplier[wk_count-2]
        else:
            neg_two = avg
        
    try:
        neg_three = final_scores[-3] * bye_multiplier[wk_count-3]
    except IndexError:
        if wk_count >= 3:
            neg_three = avg * bye_multiplier[wk_count-3]
        else:
            neg_three = avg
        
    try:
        neg_four = final_scores[-4] * bye_multiplier[wk_count-4]
    except IndexError:
        if wk_count >= 4:
            neg_four = avg * bye_multiplier[wk_count-4]
        else:
            neg_four = avg
        
    try:
        neg_five = final_scores[-5] * bye_multiplier[wk_count-5]
    except IndexError:
        if wk_count >= 5:
            neg_five = avg * bye_multiplier[wk_count-5]
        else:
            neg_five = avg
    
    plus_one = ((neg_one+neg_two+neg_three+neg_four+neg_five)/5) * (1/bye_multiplier[wk_count])
    plus_two = ((neg_one+neg_two+neg_three+neg_four+avg)/5) * (1/bye_multiplier[wk_count+1])
    plus_three = ((neg_one+neg_two+neg_three+avg+avg)/5) * (1/bye_multiplier[wk_count+2])
    plus_four = ((neg_one+neg_two+avg+avg+avg)/5) * (1/bye_multiplier[wk_count+3])
    plus_five = ((neg_one+avg+avg+avg+avg)/5) * (1/bye_multiplier[wk_count+4])
    
    for item in [plus_one, plus_two, plus_three, plus_four, plus_five]:
        final_scores.append(item)
    
    i = wk_count
    while len(final_scores) < 17:
        final_scores.append(round(avg * (1/bye_multiplier[i+5]), 3))
        i = i + 1
    
    print(final_scores)
    scores.loc[team, 'Wk1':'Wk17'] = final_scores

The Attentive Listeners
[114.5, 114.5, 114.5, 114.5, 114.5, 100.175, 93.014, 107.31, 100.175, 100.175, 107.31, 107.31, 100.175, 100.175, 114.5, 114.5, 114.5]
#CorruptCommish
[114.5, 114.5, 114.5, 114.5, 114.5, 100.175, 93.014, 107.31, 100.175, 100.175, 107.31, 107.31, 100.175, 100.175, 114.5, 114.5, 114.5]
The Paper (w/o Ja'Marr) Chase
[114.5, 114.5, 114.5, 114.5, 114.5, 100.175, 93.014, 107.31, 100.175, 100.175, 107.31, 107.31, 100.175, 100.175, 114.5, 114.5, 114.5]
The Dream Team
[114.5, 114.5, 114.5, 114.5, 114.5, 100.175, 93.014, 107.31, 100.175, 100.175, 107.31, 107.31, 100.175, 100.175, 114.5, 114.5, 114.5]
Rod the Great's Team
[114.5, 114.5, 114.5, 114.5, 114.5, 100.175, 93.014, 107.31, 100.175, 100.175, 107.31, 107.31, 100.175, 100.175, 114.5, 114.5, 114.5]
Chip Kelly
[114.5, 114.5, 114.5, 114.5, 114.5, 100.175, 93.014, 107.31, 100.175, 100.175, 107.31, 107.31, 100.175, 100.175, 114.5, 114.5, 114.5]
CTE Pros (not concussions-SQL)
[114.5, 114.5, 114.5, 114.5, 114.5, 100.175, 93.

In [454]:
scores.columns = ['Total', 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
scores #scores file

,Total,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
!Team,,,,,,,,,,,,,,,,,,
The Attentive Listeners,0,114.5,114.5,114.5,114.5,114.5,100.175,93.014,107.31,100.175,100.175,107.31,107.31,100.175,100.175,114.5,114.5,114.5
#CorruptCommish,0,114.5,114.5,114.5,114.5,114.5,100.175,93.014,107.31,100.175,100.175,107.31,107.31,100.175,100.175,114.5,114.5,114.5
The Paper (w/o Ja'Marr) Chase,0,114.5,114.5,114.5,114.5,114.5,100.175,93.014,107.31,100.175,100.175,107.31,107.31,100.175,100.175,114.5,114.5,114.5
The Dream Team,0,114.5,114.5,114.5,114.5,114.5,100.175,93.014,107.31,100.175,100.175,107.31,107.31,100.175,100.175,114.5,114.5,114.5
Rod the Great's Team,0,114.5,114.5,114.5,114.5,114.5,100.175,93.014,107.31,100.175,100.175,107.31,107.31,100.175,100.175,114.5,114.5,114.5
Chip Kelly,0,114.5,114.5,114.5,114.5,114.5,100.175,93.014,107.31,100.175,100.175,107.31,107.31,100.175,100.175,114.5,114.5,114.5
CTE Pros (not concussions-SQL),0,114.5,114.5,114.5,114.5,114.5,100.175,93.014,107.31,100.175,100.175,107.31,107.31,100.175,100.175,114.5,114.5,114.5
All Filler No Killer,0,114.5,114.5,114.5,114.5,114.5,100.175,93.014,107.31,100.175,100.175,107.31,107.31,100.175,100.175,114.5,114.5,114.5
GhostOfRayLewis,0,114.5,114.5,114.5,114.5,114.5,100.175,93.014,107.31,100.175,100.175,107.31,107.31,100.175,100.175,114.5,114.5,114.5


Using the above data frame, this function generates a score for a team given a week on the schedule.

In [471]:
def score_generator(team, week):
    try:
        mu = remaining_table.loc[team][week]
        predict = np.random.normal(loc=mu, scale=wobble)
        return(predict)
    except: #bye weeks (in playoffs)
        return(0)

In [467]:
remaining_sched = pd.read_excel('UsacOneFantasyProps2021.xlsx', sheet_name='RemainingSched') 
remaining_sched = remaining_sched[remaining_sched['Week'] > wk_count]
remaining_sched.head(12)

,Week,Team,Opponent
0,1,The Attentive Listeners,CTE Pros (not concussions-SQL)
1,1,#CorruptCommish,All Filler No Killer
2,1,The Paper (w/o Ja'Marr) Chase,GhostOfRayLewis
3,1,The Dream Team,ACE of L.A.D.
4,1,Rod the Great's Team,Catch 22
5,1,Chip Kelly,Automatic First Down
6,2,The Attentive Listeners,Rod the Great's Team
7,2,#CorruptCommish,The Paper (w/o Ja'Marr) Chase
8,2,All Filler No Killer,Automatic First Down
9,2,The Dream Team,GhostOfRayLewis


In [468]:
seed_table = pd.read_excel('UsacOneFantasyProps2021.xlsx', sheet_name='Wins + Points') 
remaining_table = scores.drop(['Total'], axis=1)

## One Loop

The below cells run through one loop of the final output; the season is played out based on the probabilities generated above and the remaining schedule.

In [390]:
remaining_sched['TeamScore'] = [score_generator(j.Team, j.Week) for i,j in remaining_sched.iterrows()]
remaining_sched['OppoScore'] = [score_generator(j.Opponent, j.Week) for i,j in remaining_sched.iterrows()]
remaining_sched['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in remaining_sched.iterrows()] #there won't be ties
remaining_sched['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in remaining_sched.iterrows()]
remaining_sched.head(6)

,Week,Team,Opponent,TeamScore,OppoScore,team_win,oppo_win
0,1,The Attentive Listeners,CTE Pros (not concussions-SQL),139.977515,109.612318,1,0
1,1,#CorruptCommish,All Filler No Killer,134.597138,61.520583,1,0
2,1,The Paper (w/o Ja'Marr) Chase,GhostOfRayLewis,99.858424,94.412201,1,0
3,1,The Dream Team,ACE of L.A.D.,101.045696,95.565382,1,0
4,1,Rod the Great's Team,Catch 22,113.130538,138.172109,0,1
5,1,Chip Kelly,Automatic First Down,60.004794,118.909629,0,1


In [391]:
#simplify that frame down to the number of wins each team gains in that simmed run of the league

wins1 = remaining_sched[['Team', 'team_win', 'TeamScore']]
wins2 = remaining_sched[['Opponent', 'oppo_win', 'OppoScore']]
wins1 = wins1.groupby('Team').sum()
wins2 = wins2.groupby('Opponent').sum()
wins2 = wins2.rename(columns={'oppo_win':'team_win'})

wins3 = pd.concat([wins1, wins2])
wins3['Team'] = wins3.index
simmed_wins = wins3.groupby('Team').sum()
simmed_wins['Points'] = simmed_wins.OppoScore + simmed_wins.TeamScore
simmed_wins = simmed_wins.drop(['OppoScore', 'TeamScore'], axis=1)

simmed_wins

,team_win,Points
Team,,
#CorruptCommish,9,1594.808564
ACE of L.A.D.,7,1571.918104
All Filler No Killer,8,1416.922791
Automatic First Down,7,1256.873502
CTE Pros (not concussions-SQL),9,1533.966822
Catch 22,8,1534.440791
Chip Kelly,6,1484.588622
GhostOfRayLewis,3,1306.868050
Rod the Great's Team,9,1655.164702


In [392]:
#read in the current standings frame

current_standings = pd.read_excel('UsacOneFantasyProps2021.xlsx', sheet_name='Wins + Points', index_col='!Team')
current_standings = current_standings[['Points', 'W']]
new_standings = current_standings
new_standings

,Points,W
!Team,,
The Attentive Listeners,0,0
#CorruptCommish,0,0
The Paper (w/o Ja'Marr) Chase,0,0
The Dream Team,0,0
Rod the Great's Team,0,0
Chip Kelly,0,0
CTE Pros (not concussions-SQL),0,0
All Filler No Killer,0,0
GhostOfRayLewis,0,0


In [393]:
#add the simmed wins to the current wins to get a simmed "outcome" of the league

for team in team_list:
    new_standings.at[team, 'W'] = new_standings.at[team, 'W'] + simmed_wins.at[team, 'team_win']


In [394]:
#rearrange the simulated outcome frame by wins and then points (the tiebreaker metric) to determine the "place" of each team
#in the simmed league

simmed_standings = current_standings.sort_values(by=['W', 'Points'], ascending=False)

#a random "points" jigger for remaining weeks to account for win ties
simmed_standings['Points'] = current_standings['Points'] + simmed_wins['Points']

simmed_standings = simmed_standings.sort_values(by=['W', 'Points'], ascending=False)
#ranking assignment
simmed_standings['place'] = np.arange(1,13)

In [395]:
simmed_standings

,Points,W,place
!Team,,,
Rod the Great's Team,1655.164702,9,1
#CorruptCommish,1594.808564,9,2
CTE Pros (not concussions-SQL),1533.966822,9,3
Catch 22,1534.440791,8,4
All Filler No Killer,1416.922791,8,5
ACE of L.A.D.,1571.918104,7,6
The Dream Team,1477.235977,7,7
Automatic First Down,1256.873502,7,8
The Attentive Listeners,1528.866310,6,9


In [396]:
dict_test = simmed_standings[['place']].to_dict()
dict2 = dict_test.pop('place')
inv_map = {v: k for k, v in dict2.items()}
inv_map

inv_map.update( {13 : 'bye'} )
inv_map.update( {14 : 'bye'} )
inv_map.update( {15 : 'bye'} )
inv_map.update( {16 : 'bye'} )

inv_map

{1: "Rod the Great's Team",
 2: '#CorruptCommish',
 3: 'CTE Pros (not concussions-SQL)',
 4: 'Catch 22',
 5: 'All Filler No Killer',
 6: 'ACE of L.A.D.',
 7: 'The Dream Team',
 8: 'Automatic First Down',
 9: 'The Attentive Listeners',
 10: 'Chip Kelly',
 11: "The Paper (w/o Ja'Marr) Chase",
 12: 'GhostOfRayLewis',
 13: 'bye',
 14: 'bye',
 15: 'bye',
 16: 'bye'}

In [397]:
matchups_r1 = {'Team': [inv_map[1], inv_map[4], inv_map[3], inv_map[2], inv_map[7],inv_map[10], inv_map[9], inv_map[8]],
        'Opponent': [inv_map[13], inv_map[5], inv_map[6], inv_map[14], inv_map[15],inv_map[11], inv_map[12], inv_map[16]]
        }

playoff_r1 = pd.DataFrame(matchups_r1, columns = ['Team', 'Opponent'])
playoff_r1

,Team,Opponent
0,Rod the Great's Team,bye
1,Catch 22,All Filler No Killer
2,CTE Pros (not concussions-SQL),ACE of L.A.D.
3,#CorruptCommish,bye
4,The Dream Team,bye
5,Chip Kelly,The Paper (w/o Ja'Marr) Chase
6,The Attentive Listeners,GhostOfRayLewis
7,Automatic First Down,bye


In [398]:
playoff_r1['TeamScore'] = [score_generator(j.Team, 15) for i,j in playoff_r1.iterrows()]
playoff_r1['OppoScore'] = [score_generator(j.Opponent, 15) for i,j in playoff_r1.iterrows()]
playoff_r1['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r1.iterrows()] #there won't be ties
playoff_r1['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r1.iterrows()]
playoff_r1.head(8)

,Team,Opponent,TeamScore,OppoScore,team_win,oppo_win
0,Rod the Great's Team,bye,86.601612,0.000000,1,0
1,Catch 22,All Filler No Killer,88.711059,113.926134,0,1
2,CTE Pros (not concussions-SQL),ACE of L.A.D.,83.290540,77.966893,1,0
3,#CorruptCommish,bye,124.013211,0.000000,1,0
4,The Dream Team,bye,187.261959,0.000000,1,0
5,Chip Kelly,The Paper (w/o Ja'Marr) Chase,96.358876,130.336321,0,1
6,The Attentive Listeners,GhostOfRayLewis,102.838607,111.683445,0,1
7,Automatic First Down,bye,82.151038,0.000000,1,0


In [399]:
winners_r1 = []
losers_r1 = []
for index, row in playoff_r1.iterrows():
    if row['team_win'] == 1:
        winners_r1.append(row['Team'])
    else:
        winners_r1.append(row['Opponent'])
        
winners_r1

["Rod the Great's Team",
 'All Filler No Killer',
 'CTE Pros (not concussions-SQL)',
 '#CorruptCommish',
 'The Dream Team',
 "The Paper (w/o Ja'Marr) Chase",
 'GhostOfRayLewis',
 'Automatic First Down']

In [400]:
matchups_r2 = {'Team': [winners_r1[0], winners_r1[2], winners_r1[4], winners_r1[6]], 
        'Opponent': [winners_r1[1], winners_r1[3], winners_r1[5], winners_r1[7]]
        }

playoff_r2 = pd.DataFrame(matchups_r2, columns = ['Team', 'Opponent'])
playoff_r2

,Team,Opponent
0,Rod the Great's Team,All Filler No Killer
1,CTE Pros (not concussions-SQL),#CorruptCommish
2,The Dream Team,The Paper (w/o Ja'Marr) Chase
3,GhostOfRayLewis,Automatic First Down


In [401]:
playoff_r2['TeamScore'] = [score_generator(j.Team, 16) for i,j in playoff_r2.iterrows()]
playoff_r2['OppoScore'] = [score_generator(j.Opponent, 16) for i,j in playoff_r2.iterrows()]
playoff_r2['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r2.iterrows()] #there won't be ties
playoff_r2['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r2.iterrows()]
playoff_r2.head(4)

,Team,Opponent,TeamScore,OppoScore,team_win,oppo_win
0,Rod the Great's Team,All Filler No Killer,125.043053,111.801794,1,0
1,CTE Pros (not concussions-SQL),#CorruptCommish,113.338876,109.749203,1,0
2,The Dream Team,The Paper (w/o Ja'Marr) Chase,117.721153,75.521664,1,0
3,GhostOfRayLewis,Automatic First Down,126.206363,107.082580,1,0


In [402]:
winners_r2 = []
losers_r2 = []
for index, row in playoff_r2.iterrows():
    if row['team_win'] == 1:
        winners_r2.append(row['Team'])
        losers_r2.append(row['Opponent'])
    else:
        winners_r2.append(row['Opponent'])
        losers_r2.append(row['Team'])
        
winners_r2

["Rod the Great's Team",
 'CTE Pros (not concussions-SQL)',
 'The Dream Team',
 'GhostOfRayLewis']

In [403]:
matchups_r3 = {'Team': [winners_r2[0], losers_r2[0], winners_r2[2]], 
        'Opponent': [winners_r2[1], losers_r2[1], winners_r2[3]]
        }

playoff_r3 = pd.DataFrame(matchups_r3, columns = ['Team', 'Opponent'])
playoff_r3

,Team,Opponent
0,Rod the Great's Team,CTE Pros (not concussions-SQL)
1,All Filler No Killer,#CorruptCommish
2,The Dream Team,GhostOfRayLewis


In [404]:
playoff_r3['TeamScore'] = [score_generator(j.Team, 17) for i,j in playoff_r3.iterrows()]
playoff_r3['OppoScore'] = [score_generator(j.Opponent, 17) for i,j in playoff_r3.iterrows()]
playoff_r3['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r3.iterrows()] #there won't be ties
playoff_r3['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r3.iterrows()]
playoff_r3.head(3)

,Team,Opponent,TeamScore,OppoScore,team_win,oppo_win
0,Rod the Great's Team,CTE Pros (not concussions-SQL),90.129048,105.714121,0,1
1,All Filler No Killer,#CorruptCommish,130.628025,116.777046,1,0
2,The Dream Team,GhostOfRayLewis,142.508603,168.838508,0,1


In [405]:
winners_r3 = []
losers_r3 = []
for index, row in playoff_r3.iterrows():
    if row['team_win'] == 1:
        winners_r3.append(row['Team'])
        losers_r3.append(row['Opponent'])
    else:
        winners_r3.append(row['Opponent'])
        losers_r3.append(row['Team'])
        
winners_r3

['CTE Pros (not concussions-SQL)', 'All Filler No Killer', 'GhostOfRayLewis']

In [406]:
champ_table = pd.DataFrame(columns=['First', 'Second', 'Third', 'Consolation'])

matchups = []
for index, row in champ_table.iterrows():
    championship = []
    championship.append(row['First'])
    championship.append(row['Second'])
    championship = sorted(championship)
    matchups.append(str(championship))
    
counter=collections.Counter(matchups)
counter

champ_table = champ_table.append({'First' : winners_r3[0], 'Second' : losers_r3[0], 'Third' : winners_r3[1], 'Consolation' : winners_r3[2] } , ignore_index=True)

champ_table

,First,Second,Third,Consolation
0,CTE Pros (not concussions-SQL),Rod the Great's Team,All Filler No Killer,GhostOfRayLewis


## Does The Deal

Runs the above loop *number_sims* times, usually 10,000 (~30 minute runtime).

In [440]:
#this is the equivalent of looping the previous five code blocks n number of times to get a distribution of possible outcomes
#for each team

all_predictions = pd.DataFrame(columns=['Points', 'W', 'place'])
i = 0

number_sims = 1000 #takes about 60 seconds per thousand sims on my machine
seed_table = pd.read_excel('UsacOneFantasyProps2021.xlsx', sheet_name='Wins + Points') 

current_standings = pd.read_excel('UsacOneFantasyProps2021.xlsx', sheet_name='Wins + Points', index_col='!Team')

champ_table = pd.DataFrame(columns=['First', 'Second', 'Third', 'Consolation'])

while i < number_sims: 
    #game sims
    
    remaining_table = scores.drop(['Total'], axis=1) #automate this
    
    remaining_sched['TeamScore'] = [score_generator(j.Team, j.Week) for i,j in remaining_sched.iterrows()]
    remaining_sched['OppoScore'] = [score_generator(j.Opponent, j.Week) for i,j in remaining_sched.iterrows()]
    remaining_sched['team_win'] = [int(j.TeamScore>j.OppoScore) for i,j in remaining_sched.iterrows()]
    remaining_sched['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in remaining_sched.iterrows()]
    
    wins1 = remaining_sched[['Team', 'team_win', 'TeamScore']]
    wins2 = remaining_sched[['Opponent', 'oppo_win', 'OppoScore']]
    wins1 = wins1.groupby('Team').sum()
    wins2 = wins2.groupby('Opponent').sum()
    wins2 = wins2.rename(columns={'oppo_win':'team_win'})

    wins3 = pd.concat([wins1, wins2])
    wins3['Team'] = wins3.index
    simmed_wins = wins3.groupby('Team').sum()
    simmed_wins['Points'] = simmed_wins.OppoScore + simmed_wins.TeamScore
    simmed_wins = simmed_wins.drop(['OppoScore', 'TeamScore'], axis=1)
    
    new_standings = current_standings[['Points', 'W']]
    
    for team in team_list:
        new_standings.at[team, 'W'] = new_standings.at[team, 'W'] + simmed_wins.at[team, 'team_win']
        
    #rearrange the simulated outcome frame by wins and then points (the tiebreaker metric) to determine the "place" of each team
    #in the simmed league

    simmed_standings = new_standings.sort_values(by=['W', 'Points'], ascending=False)

    #a random "points" jigger for remaining weeks to account for win ties
    simmed_standings['Points'] = new_standings['Points'] + simmed_wins['Points']

    simmed_standings = simmed_standings.sort_values(by=['W', 'Points'], ascending=False)
    #ranking assignment
    simmed_standings['place'] = np.arange(1,13)
    
    all_predictions = pd.concat([all_predictions, simmed_standings])

    i = i + 1

    #playoff sim
    
    dict_test = simmed_standings[['place']].to_dict()
    dict2 = dict_test.pop('place')
    inv_map = {v: k for k, v in dict2.items()}

    inv_map.update( {13 : 'bye'} )
    inv_map.update( {14 : 'bye'} )
    inv_map.update( {15 : 'bye'} )
    inv_map.update( {16 : 'bye'} )
    
    ##ROUND ONE
    
    matchups_r1 = {'Team': [inv_map[1], inv_map[4], inv_map[3], inv_map[2], inv_map[7],inv_map[10], inv_map[9], inv_map[8]],
        'Opponent': [inv_map[13], inv_map[5], inv_map[6], inv_map[14], inv_map[15],inv_map[11], inv_map[12], inv_map[16]]
        }

    playoff_r1 = pd.DataFrame(matchups_r1, columns = ['Team', 'Opponent'])
    playoff_r1
    
    playoff_r1['TeamScore'] = [score_generator(j.Team, 15) for i,j in playoff_r1.iterrows()]
    playoff_r1['OppoScore'] = [score_generator(j.Opponent, 15) for i,j in playoff_r1.iterrows()]
    playoff_r1['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r1.iterrows()] #there won't be ties
    playoff_r1['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r1.iterrows()]
    
    winners_r1 = []
    losers_r1 = []
    for index, row in playoff_r1.iterrows():
        if row['team_win'] == 1:
            winners_r1.append(row['Team'])
        else:
            winners_r1.append(row['Opponent'])
            
    ##ROUND TWO
    
    matchups_r2 = {'Team': [winners_r1[0], winners_r1[2], winners_r1[4], winners_r1[6]], 
        'Opponent': [winners_r1[1], winners_r1[3], winners_r1[5], winners_r1[7]]
        }

    playoff_r2 = pd.DataFrame(matchups_r2, columns = ['Team', 'Opponent'])
    playoff_r2
    
    playoff_r2['TeamScore'] = [score_generator(j.Team, 16) for i,j in playoff_r2.iterrows()]
    playoff_r2['OppoScore'] = [score_generator(j.Opponent, 16) for i,j in playoff_r2.iterrows()]
    playoff_r2['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r2.iterrows()] #there won't be ties
    playoff_r2['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r2.iterrows()]
    
    winners_r2 = []
    losers_r2 = []
    for index, row in playoff_r2.iterrows():
        if row['team_win'] == 1:
            winners_r2.append(row['Team'])
            losers_r2.append(row['Opponent'])
        else:
            winners_r2.append(row['Opponent'])
            losers_r2.append(row['Team'])
            
    ##ROUND THREE
    
    matchups_r3 = {'Team': [winners_r2[0], losers_r2[0], winners_r2[2]], 
        'Opponent': [winners_r2[1], losers_r2[1], winners_r2[3]]
        }

    playoff_r3 = pd.DataFrame(matchups_r3, columns = ['Team', 'Opponent'])
    
    playoff_r3['TeamScore'] = [score_generator(j.Team, 17) for i,j in playoff_r3.iterrows()]
    playoff_r3['OppoScore'] = [score_generator(j.Opponent, 17) for i,j in playoff_r3.iterrows()]
    playoff_r3['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r3.iterrows()] #there won't be ties
    playoff_r3['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r3.iterrows()]
    
    winners_r3 = []
    losers_r3 = []
    for index, row in playoff_r3.iterrows():
        if row['team_win'] == 1:
            winners_r3.append(row['Team'])
            losers_r3.append(row['Opponent'])
        else:
            winners_r3.append(row['Opponent'])
            losers_r3.append(row['Team'])

    matchups = []
    
    for index, row in champ_table.iterrows():
        championship = []
        championship.append(row['First'])
        championship.append(row['Second'])
        championship = sorted(championship)
        matchups.append(str(championship))

    counter=collections.Counter(matchups)

    champ_table = champ_table.append({'First' : winners_r3[0], 'Second' : losers_r3[0], 'Third' : winners_r3[1], 'Consolation' : winners_r3[2] } , ignore_index=True)

    if (i % 100 == 0):
        print(str(i/100) + '%') #totally unnecessary, just gives a sense of progress

1.0%
2.0%
3.0%
4.0%
5.0%
6.0%
7.0%
8.0%
9.0%
10.0%


## Aggregation of Sims for Final Figures

In [442]:
#table of distribution's of each team's simmed outcomes

all_predictions['team'] = all_predictions.index
outcome_dist = pd.pivot_table(all_predictions, values = ['place'], index = ['team'], columns=all_predictions.place.values, aggfunc='count', fill_value=0)

In [443]:
all_predictions['W2'] = [float(i) for i in all_predictions.W]
all_predictions['team'] = all_predictions.index
average_wins = pd.DataFrame(all_predictions.groupby('team')['W2'].mean())

In [444]:
#season-ending placement probs

outcome_dist['Playoffs'] = (outcome_dist.place[1] + outcome_dist.place[2] 
                            + outcome_dist.place[3] + outcome_dist.place[4]
                            + outcome_dist.place[5] + outcome_dist.place[6]) / number_sims
outcome_dist['FRB'] = (outcome_dist.place[1] + outcome_dist.place[2]) / number_sims

outcome_dist['first'] = (outcome_dist.place[1]) / number_sims
outcome_dist['second'] = (outcome_dist.place[2]) / number_sims
outcome_dist['third'] = (outcome_dist.place[3]) / number_sims
outcome_dist['fourth'] = (outcome_dist.place[4]) / number_sims
outcome_dist['fifth'] = (outcome_dist.place[5]) / number_sims
outcome_dist['sixth'] = (outcome_dist.place[6]) / number_sims
outcome_dist['seventh'] = (outcome_dist.place[7]) / number_sims
outcome_dist['eighth'] = (outcome_dist.place[8]) / number_sims
outcome_dist['ninth'] = (outcome_dist.place[9]) / number_sims
outcome_dist['tenth'] = (outcome_dist.place[10]) / number_sims
outcome_dist['eleventh'] = (outcome_dist.place[11]) / number_sims
outcome_dist['twelfth'] = (outcome_dist.place[12]) / number_sims


outcome_dist = outcome_dist.sort_values(['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'tenth', 'eleventh', 'twelfth'], ascending=False) #after 20
outcome2 = outcome_dist.drop(['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'tenth', 'eleventh', 'twelfth'], axis=1) 
outcome3 = pd.merge(outcome2, average_wins, left_index=True, right_on='team')

outcome3.columns = ['Playoffs', 'FRB',1,2,3,4,5,6,7,8,9,10,11,12,'Wins']
points_col = all_predictions.groupby('team').mean()
#outcome3['AvgPoints'] = round(points_col.Points, 2)
outcome3

c:\users\zachary.gozlan\appdata\local\programs\python\python39\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,Playoffs,FRB,1,2,3,4,5,6,7,8,9,10,11,12,Wins
team,,,,,,,,,,,,,,,
Rod the Great's Team,99,88,83,85,93,61,76,85,73,66,97,94,0.509,0.187,7.051
All Filler No Killer,96,74,90,95,76,91,89,88,78,63,94,66,0.522,0.170,7.128
Chip Kelly,91,66,95,86,82,81,88,86,74,94,76,81,0.501,0.157,6.996
CTE Pros (not concussions-SQL),90,83,80,81,83,86,72,80,77,92,91,85,0.503,0.173,7.027
Automatic First Down,83,93,70,76,83,98,79,92,97,75,70,84,0.503,0.176,7.023
ACE of L.A.D.,81,92,66,85,83,74,83,96,78,78,84,100,0.481,0.173,6.935
The Paper (w/o Ja'Marr) Chase,80,87,93,75,93,75,93,68,78,80,82,96,0.503,0.167,7.011
The Dream Team,78,89,81,80,86,80,79,87,88,95,82,75,0.494,0.167,6.983
The Attentive Listeners,78,82,85,89,70,81,77,81,90,81,93,93,0.485,0.160,6.917


In [445]:
#playoff probs

first_table = pd.DataFrame(champ_table['First'].value_counts())
second_table = pd.DataFrame(champ_table['Second'].value_counts())
third_table = pd.DataFrame(champ_table['Third'].value_counts())
consolation_table = pd.DataFrame(champ_table['WinConsolation'].value_counts())

dfs = [first_table, second_table, third_table, consolation_table]

final = first_table.merge(second_table, how='outer', on=None, left_index=True, right_index=True)
final = final.merge(third_table, how='outer', on=None, left_index=True, right_index=True)
final = final.merge(consolation_table, how='outer', on=None, left_index=True, right_index=True)
final = final.fillna(0).sort_values(['First', 'Second', 'Third', 'WinConsolation'], ascending=False)
#final['value'] = round((((final.First/10000)*325) + ((final.Second/10000)*175) + ((final.Third/10000)*50) + ((final.Consolation/10000)*50)),2)
final

,First,Second,Third,Consolation
All Filler No Killer,100,71,92,77
GhostOfRayLewis,93,78,79,88
Rod the Great's Team,91,86,84,73
CTE Pros (not concussions-SQL),90,84,89,96
The Attentive Listeners,85,87,81,85
The Paper (w/o Ja'Marr) Chase,82,86,76,90
#CorruptCommish,80,88,86,70
Catch 22,78,92,85,84
Chip Kelly,78,87,86,81
ACE of L.A.D.,77,85,77,86


In [441]:
counter #friviolity of friviloties: most common finals matchups in sims

Counter({"['#CorruptCommish', 'Catch 22']": 15,
         '[\'#CorruptCommish\', "Rod the Great\'s Team"]': 20,
         '["Rod the Great\'s Team", "The Paper (w/o Ja\'Marr) Chase"]': 18,
         "['ACE of L.A.D.', 'CTE Pros (not concussions-SQL)']": 14,
         "['All Filler No Killer', 'Catch 22']": 20,
         '[\'ACE of L.A.D.\', "The Paper (w/o Ja\'Marr) Chase"]': 9,
         '[\'GhostOfRayLewis\', "Rod the Great\'s Team"]': 18,
         "['Automatic First Down', 'Chip Kelly']": 15,
         '[\'Catch 22\', "Rod the Great\'s Team"]': 15,
         '[\'CTE Pros (not concussions-SQL)\', "Rod the Great\'s Team"]': 15,
         "['#CorruptCommish', 'All Filler No Killer']": 13,
         "['Chip Kelly', 'GhostOfRayLewis']": 17,
         "['Chip Kelly', 'The Dream Team']": 12,
         "['ACE of L.A.D.', 'Catch 22']": 19,
         "['Catch 22', 'GhostOfRayLewis']": 17,
         "['Automatic First Down', 'GhostOfRayLewis']": 14,
         "['#CorruptCommish', 'ACE of L.A.D.']": 15,
     

In [368]:
outcome3.to_csv('UsacOne_2021_season_' + wk_count + '.csv')
final.to_csv('UsacOne_2021_playoffs_' + wk_count + '.csv')

## Previous Weeks